In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle'): #/input
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dtypes_df = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2022/train_dtypes.csv')
dtypes = {k: v for (k, v) in zip(dtypes_df.column, dtypes_df.dtype)}
train0_df = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2022/train_0.csv', dtype=dtypes)
train0_df

In [ ]:
#train_df = train0_df

#for i in range(1,10):
#    temp_df = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2022/train_{0}.csv'.format(i), dtype=dtypes)
#    train_df = pd.concat([train_df,temp_df])
#train_df

In [ ]:
train0_df[200:230]

In [ ]:
def no_team_scoring(team_A,team_B):
    if team_A == 0 and team_B == 0:
        return 1
    else:
        return 0
    
train0_df['no_team_scoring_within_10sec'] = train0_df.apply(lambda x: no_team_scoring(x['team_A_scoring_within_10sec'],x['team_B_scoring_within_10sec']), axis=1)

In [ ]:
train0_df[200:230]

In [ ]:
train0_df.describe()

In [ ]:
train0_df['team_A_scoring_within_10sec'].mean() + train0_df['team_B_scoring_within_10sec'].mean() + train0_df['no_team_scoring_within_10sec'].mean()

In [ ]:
train0_df.columns

In [ ]:
train0_df['team_scoring_next'].value_counts()

# Define train and test sets

In [ ]:
target = ['team_A_scoring_within_10sec', 'team_B_scoring_within_10sec', 'no_team_scoring_within_10sec']
#target = ['team_A_scoring_within_10sec']
y = train0_df[target]
y.shape

In [ ]:
features = train0_df.columns[3:-5]

In [ ]:
features

In [ ]:
#features2 = ['ball_pos_x', 'ball_pos_y', 'ball_pos_z', 'ball_vel_x', 'ball_vel_y','ball_vel_z']
X = train0_df[features]
X.shape

In [ ]:
X.isnull().any()

In [ ]:
X.fillna(0, inplace=True)

In [ ]:
X.isnull().any()

In [ ]:
from sklearn.model_selection import train_test_split
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1, test_size=0.1)

In [ ]:
print ("number of training examples = " + str(train_X.shape[0]))
print ("number of test examples = " + str(val_X.shape[0]))
print ("train_X shape: " + str(train_X.shape))
print ("train_y shape: " + str(train_y.shape))
print ("val_X shape: " + str(val_X.shape))
print ("val_y shape: " + str(val_y.shape))

# Define sequential model

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as tfl

In [ ]:
def simple_model():
    """
    Implements the forward propagation for the binary? classification model:
    DENSE -> RELU -> DENSE -> SIGMOID
    
    Arguments:
    None

    Returns:
    model -- TF Keras model (object containing the information for the entire training process) 
    """
    model = tf.keras.Sequential([
        tfl.Dense(16, input_shape=(train_X.shape[1],)),
        tfl.Dense(3, activation='sigmoid')
        ])
    
    return model

In [ ]:
# 1st way: use defined function
#model = simple_model()

In [ ]:
# 2nd way: use model add
model = tf.keras.Sequential()
model.add(tfl.Input(shape=(train_X.shape[1],)))
model.add(tfl.Dense(256, activation='relu'))
#model.add(tfl.Dense(256, input_shape=(train_X.shape[1],)))
model.add(tfl.Dense(256, activation='relu'))
model.add(tfl.Dense(256, activation='relu'))
model.add(tfl.Dense(256, activation='relu'))
model.add(tfl.Dense(256, activation='relu'))
model.add(tfl.Dense(256, activation='relu'))
model.add(tfl.Dense(256, activation='relu'))
model.add(tfl.Dense(256, activation='relu'))
model.add(tfl.Dense(256, activation='relu'))
model.add(tfl.Dense(256, activation='relu'))
model.add(tfl.Dense(256, activation='relu'))
model.add(tfl.Dense(256, activation='relu'))
model.add(tfl.Dense(256, activation='relu'))
model.add(tfl.Dense(3, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])
model.summary()

# Train model

In [ ]:
history = model.fit(x=train_X, y=train_y,
                    epochs=30, batch_size=16,
                    validation_data=(val_X, val_y))

In [ ]:
model.save('/kaggle/working/trained_model_large')
model.save_weights('/kaggle/working/trained_weights_large')

In [ ]:
#model.fit(train_X, train_y, epochs=5, batch_size=16)

In [ ]:
model.evaluate(val_X, val_y)

In [ ]:
# The history.history["loss"] entry is a dictionary with as many values as epochs that the
# model was trained on. 
df_loss_acc = pd.DataFrame(history.history)
df_loss= df_loss_acc[['loss','val_loss']]
df_loss.rename(columns={'loss':'train','val_loss':'validation'},inplace=True)
df_acc= df_loss_acc[['accuracy','val_accuracy']]
df_acc.rename(columns={'accuracy':'train','val_accuracy':'validation'},inplace=True)
df_loss.plot(title='Model loss',figsize=(12,8)).set(xlabel='Epoch',ylabel='Loss')
df_acc.plot(title='Model Accuracy',figsize=(12,8)).set(xlabel='Epoch',ylabel='Accuracy')

# Predict over test set

In [ ]:
dtypes_df = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2022/test_dtypes.csv')
dtypes = {k: v for (k, v) in zip(dtypes_df.column, dtypes_df.dtype)}
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2022/test.csv', dtype=dtypes)
test_df

In [ ]:
sample_sub_df = pd.read_csv('/kaggle/input/tabular-playground-series-oct-2022/sample_submission.csv', dtype=dtypes)
sample_sub_df

In [ ]:
X_test = test_df[features]
X_test.shape

In [ ]:
y_test = model.predict(X_test)
y_test

In [ ]:
y_test.shape

In [ ]:
my_sub_df = pd.DataFrame({'Id': test_df.id,
                          'team_A_scoring_within_10sec': y_test[:, 0],
                          'team_B_scoring_within_10sec': y_test[:, 1]})
my_sub_df

In [ ]:
my_sub_df.fillna(0, inplace=True)

In [ ]:
my_sub_df.to_csv('submission.csv', index=False)